In [182]:
import pandas as pd
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_rows', 500)

import re

In [183]:
import datetime

In [184]:
import numpy as np

In [185]:
df = pd.read_csv("pl_resume_dirty.csv")

In [186]:
#df.head()
#df.shape
# df.dtypes

### Clean name and get a rid of duplicates

In [187]:
df["name"] = df["name"].str.strip()
df["name"] = df["name"].str.replace("同志简历" , "")
df["name"] = df["name"].str.replace("简历" , "")

In [188]:
df[(df.name == "王东明") | (df.name == "王晨")].shape

(80, 4)

In [189]:
df[df.name == "王东明"].index

Int64Index([ 703,  704,  705,  706,  707,  708,  709,  710,  711,  712,  713,
             714,  715,  716,  717,  718,  719,  720,  721,  722,  723, 1435,
            1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446,
            1447, 1448, 1449, 1450, 1451, 1452, 1453],
           dtype='int64')

In [190]:
df = df.drop([1435,1436, 1437, 1438, 1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448, 1449, 1450, 1451, 1452, 1453])

In [191]:
df[df.name == "王晨"].index

Int64Index([171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
            184, 185, 186, 187, 188, 189, 190, 565, 566, 567, 568, 569, 570,
            571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583,
            584],
           dtype='int64')

In [192]:
df = df.drop([565, 566, 567, 568, 569, 570, 571, 572, 573, 574, 575, 576, 577, 578, 579, 580, 581, 582, 583, 584])

In [193]:
df.shape

(1415, 4)

In [194]:
df[(df.name == "王东明") | (df.name == "王晨")].shape

(41, 4)

### Take a look at columns that doesn't have data for year

In [195]:
#df[df.year.isnull()]

In [196]:
#df[df.year.isnull()].shape

### Among these 81 rows that doesn't have info for year, some of them are random data that got in there, I don't want those

### others are their current positition ---> that's what I want

In [197]:
df["year"] = df["year"].fillna(0)

In [198]:
def get_year_from_content(row):
    if row["year"] == 0:
        yr = re.findall("([2019]{2}\d\d)[—－―]", row["content"])
        yr = "".join(yr)
    else:
        yr = row["year"]
    return yr

In [199]:
df["year"] = df.apply(get_year_from_content, axis=1)

In [200]:
df["year"] = df["year"].str.strip()

In [201]:
df["year"] = df["year"].str.replace("（", "")

In [202]:
#df[df.year == ""].index

In [203]:
df = df.drop([245, 830, 831, 856, 960, 961, 1394, 1413, 1414, 1434, 1433])

In [204]:
# df[df.year == ""]

### Now looking at the year column, I notice that almost all of them are continious

### So I'm going to make a very irresponsible assumption --- perhaps ALL of them are contious, meaning I can drop the year after "-" and assume whoever that person is held the same position until year in the next row

### But just to be safe, I will save this into a new column, incase I want to revisit the raw data later

In [205]:
#df.shape

In [206]:
df["year_clean"] = df["year"].str.extract(r"(^[2019]{2}\d\d)").astype(int)

### Now onto the content part

#### There are lots information here, but for the sake of this project, I will focus on **location**
#### Here are a few ways to determine the location:
#### 1. If the string has “省 ” (province), “特别行政区”(Special administrative region like Hong Kong), “自治区”(autonomous region like Tibet and Uighur) or “市” (city) in it, then grab 3 characters that comes before it
#### 2. Elif the string has “中央”(central), “最高”(highest), “国家” or “全国”(both mean national) in it, then the location is Beijing, because that's where central government and highest courts are
#### 3. Elif the string has “校” or “学”(both mean school/university) in it, then grab what's in front of it, because that means that person was in school during those years, and we can google the location
#### 4. Elif the string has “驻” in it, then grab everything after it because it means this person was a diplomat in another country
#### 5. Elif the string has “军”(army) in it, it means that person was in PLA.
#### 6. Elif the string has "国务院"(state council) in it, it's most def in Beijing
#### 7. Otherwise just grab the first 5 characters

In [207]:
df["content"] = df["content"].str.strip()
df["content"] = df["content"].str.replace("（", "")
df["content"] = df["content"].str.replace("）", "")
df["content"] = df["content"].str.replace("；", " ")
df["content"] = df["content"].str.replace("：", "")
df["content"] = df["content"].str.replace("—", "")
df["content"] = df["content"].str.replace(".", "")
df["content"] = df["content"].str.replace("-", "")
df["content"] = df["content"].str.replace(r"[0-9]{0,}", "")

In [229]:
def get_location(row):
    resume = row["content"].split("、")

    if "北京" in resume[0]:
        location = "北京"
    elif "上海" in resume[0]:
        location = "上海"
    elif "天津" in resume[0]:
        location = "天津"
    elif "重庆" in resume[0]:
        location = "重庆"
    elif "河北" in resume[0]:
        location = "河北"
    elif "河南" in resume[0]:
        location = "河南"
    elif "湖北" in resume[0]:
        location = "湖北"        
    elif "湖南" in resume[0]:
        location = "湖南"
    elif "江苏" in resume[0]:
        location = "江苏"    
    elif "江西" in resume[0]:
        location = "江西"
    elif "辽宁" in resume[0]:
        location = "辽宁"
    elif "吉林" in resume[0]:
        location = "吉林"
    elif "黑龙江" in resume[0]:
        location = "黑龙江"
    elif "陕西" in resume[0]:
        location = "陕西"
    elif "黑龙江" in resume[0]:
        location = "黑龙江"
    elif "山东" in resume[0]:
        location = "山东"
    elif "山西" in resume[0]:
        location = "山西"
    elif "四川" in resume[0]:
        location = "四川"
    elif "青海" in resume[0]:
        location = "青海"
    elif "安徽" in resume[0]:
        location = "安徽"
    elif "海南" in resume[0]:
        location = "海南"
    elif "广东" in resume[0]:
        location = "广东"
    elif "贵州" in resume[0]:
        location = "贵州"
    elif "广东" in resume[0]:
        location = "广东"
    elif "浙江" in resume[0]:
        location = "浙江"
    elif "福建" in resume[0]:
        location = "福建"
    elif "台湾" in resume[0]:
        location = "台湾"
    elif "甘肃" in resume[0]:
        location = "甘肃"
    elif "云南" in resume[0]:
        location = "云南"
    elif "内蒙古" in resume[0]:
        location = "内蒙古"
    elif "宁夏" in resume[0]:
        location = "宁夏"
    elif "新疆维吾尔" in resume[0]:
        location = "新疆维吾尔"
    elif "西藏" in resume[0]:
        location = "西藏"
    elif "广西" in resume[0]:
        location = "广西"
    elif "香港" in resume[0] or "港" in resume[0]:
        location = "香港"
    elif "澳门" in resume[0]:
        location = "澳门"
    elif "美国" in resume[0]:
        location = "美国"
    elif "英国" in resume[0]:
        location = "英国"
    elif "加拿大" in resume[0]:
        location = "加拿大"
    elif "苏州" in resume[0]:
        location = "江苏"
    
    else:
        if "市" in resume[0]:
            loc_list = re.findall("([^、，\d在]{2}市)", resume[0])
            location = "".join(loc_list)
        elif "县" in resume[0]:
            loc_list = re.findall("([^、，\d在]{1,}县)", resume[0])
            location = "".join(loc_list)
        elif "中央" in resume[0]:
            location = "北京"
        elif "军" in resume[0]:
            loc_list = re.findall("([陆海空放]{1}军)", row["content"])
            location = "".join(loc_list)
        elif "大学" in resume[0] or "学校" in resume[0] or "学院" in resume[0]:
            loc_list = re.findall("([^、，\d系在年获]{3,8}[学校院])", resume[0])
            location = "".join(loc_list)
        elif "驻" in resume[0] and "使" in resume[0]:
            loc_list = re.findall("驻(.*)使", resume[0])
            location = "".join(loc_list)
        elif "国务院" in resume[0]:
            location = "北京"
        elif "中央党校" in resume[0]:
            location = "北京"
        elif "最高" in resume[0]:
            location = "北京"
        elif "部" in resume[0] or "中国" in resume[0] or "全国" in resume[0] or "国家" in resume[0]:
            location = "北京"
        else:
            loc_list = re.findall("([^，\-.。]{3})", resume[0])
            location = "".join(loc_list)
    return location

In [209]:
df["location"] = df.apply(get_location, axis=1)

In [210]:
df.at[384, "location"] = "辽宁"
df.at[395, "location"] = "北京"

In [211]:
df.location = df.location.str.replace("清华大学","北京")
df.location = df.location.str.replace("复旦大学","上海")
df.location = df.location.str.replace("武汉大学","湖北")
df.location = df.location.str.replace("中国人民大学","北京")
df.location = df.location.str.replace("放军","解放军")
df.location = df.location.str.replace("中国科学院","北京")
df.location = df.location.str.replace("华东政法学院","上海")
df.location = df.location.str.replace("国务委员","北京")
df.location = df.location.str.replace("人民日报社社","北京")
df.location = df.location.str.replace("同济大学","上海")
df.location = df.location.str.replace("中国社会科学院","北京")
df.location = df.location.str.replace("武汉大学","湖北")
df.location = df.location.str.replace("东北重型机械学院压工艺及设备专业学","河北")
df.location = df.location.str.replace("放军陆军","解放军")
df.location = df.location.str.replace("陆军","解放军")
df.location = df.location.str.replace("空军","解放军")
df.location = df.location.str.replace("海军","解放军")
df.location = df.location.str.replace("国防大学","北京")
df.location = df.location.str.replace("清华大学基本有机合成专业学","北京")
df.location = df.location.str.replace("南昌大学人文学院","江西")
df.location = df.location.str.replace("国务委","北京")
df.location = df.location.str.replace("绍兴师范专科学校","浙江")
df.location = df.location.str.replace("－　北京","北京")
df.location = df.location.str.replace("北京基本有机合成专业学","北京")
df.location = df.location.str.replace("东北林业大学","黑龙江")
df.location = df.location.str.replace("中国农业大学","北京")
df.location = df.location.str.replace("解解放军","解放军")
df.location = df.location.str.replace("北京地质研究所古生物学及地层学","北京")
df.location = df.location.str.replace("福州大学","福建")
df.location = df.location.str.replace("人民日报社总","北京")
df.location = df.location.str.replace("人民日报社编辑委员会委员","北京")
df.location = df.location.str.replace("首都师范大学","北京")
df.location = df.location.str.replace("湖北法学院","湖北")
df.location = df.location.str.replace("哈尔滨工业大学","黑龙江")
df.location = df.location.str.replace("兼任中国医学科学院","北京")
df.location = df.location.str.replace("郑州大学","河南")
df.location = df.location.str.replace("第二炮兵五十四基地参谋长","解放军")
df.location = df.location.str.replace("佳木斯医学院","黑龙江")

In [212]:
df.location = df.location.str.replace("色拉寺","西藏")
df.location = df.location.str.replace("德国海德堡马普研究所访问","德国")
df.location = df.location.str.replace("第二炮兵五十四基地八一三旅副参","解放军")
df.location = df.location.str.replace("齐鲁石化公司团委副","山东")
df.location = df.location.str.replace("西北大学历史专业学","陕西")
df.location = df.location.str.replace("北京编室编","北京")
df.location = df.location.str.replace("湖北法学博士学","湖北")
df.location = df.location.str.replace("克劳斯塔尔工业大学","德国")
df.location = df.location.str.replace("北京人文社会学职研究生班学法学博士学","北京")
df.location = df.location.str.replace("南开大学汉语言文学专业学","天津")
df.location = df.location.str.replace("哈尔滨建筑工程学院业与民用建筑专业学","黑龙江")
df.location = df.location.str.replace("湖北经济学专业学","湖北")
df.location = df.location.str.replace("北京生物学","北京")
df.location = df.location.str.replace("第二炮兵五十四基地八一三团通信连战士","解放军")
df.location = df.location.str.replace("北京一分校国民经济计划专业学","北京")
df.location = df.location.str.replace("第二炮兵五十四基地八一三团发射一营一连排长其间－","解放军")
df.location = df.location.str.replace("第二炮兵五十四基地八一三旅旅长其间－","解放军")
df.location = df.location.str.replace("汽车工业高等专科学","吉林")
df.location = df.location.str.replace("北京副校","北京")
df.location = df.location.str.replace("台湾","台湾")
df.location = df.location.str.replace("第二炮兵五十三基地司令员","解放军")
df.location = df.location.str.replace("解放军解放军","解放军")
df.location = df.location.str.replace("南昌大学副校","江西")
df.location = df.location.str.replace("华东工程学院自动武器设计专业学","黑龙江")
df.location = df.location.str.replace("南昌大学","江西")
df.location = df.location.str.replace("上海校","上海")
df.location = df.location.str.replace("西南政法学院研究生院","重庆")
df.location = df.location.str.replace("上海经济学政治经济学研究生课程进修班学经济学硕士学","上海")
df.location = df.location.str.replace("上海法学院院","上海")
df.location = df.location.str.replace("青岛化工学院","山东")
df.location = df.location.str.replace("银行市","北京")
df.location = df.location.str.replace("厦门大学职研究生学经济学博士学","福建")
df.location = df.location.str.replace("齐鲁石化公司党委副","山东")
df.location = df.location.str.replace("北京化学与化学催化动力学","北京")
df.location = df.location.str.replace("冬奥会和冬残奥会组织委员会主席","北京")
df.location = df.location.str.replace("北京研究生院","北京")
df.location = df.location.str.replace("航天总公司二三〇厂常务副","北京")
df.location = df.location.str.replace("上海法律专业学","上海")
df.location = df.location.str.replace("日本","日本")
df.location = df.location.str.replace("－ 北京","北京")
df.location = df.location.str.replace("厦门大学财政金融专业学","福建")
df.location = df.location.str.replace("南京大学历史学职研究生学历史学博士学","江苏")
df.location = df.location.str.replace("上海光测力学","上海")
df.location = df.location.str.replace("上海上海校","上海")
df.location = df.location.str.replace("－ 中华人民共和国副主席","北京")
df.location = df.location.str.replace("华东师范大学夜大学政教专业学","上海")
df.location = df.location.str.replace("华东师范大学职研究生学经济学硕士学","上海")
df.location = df.location.str.replace("第二炮兵参谋","解放军")
df.location = df.location.str.replace("第二炮兵五十四基地八一三旅参谋","解放军")
df.location = df.location.str.replace("海南","海南")
df.location = df.location.str.replace("北京公共管理学院公共管理专业学管理学博士学","北京")
df.location = df.location.str.replace("谋长其间－北京战略指挥培训班学","解放军")
df.location = df.location.str.replace("第二炮兵司令","解放军")
df.location = df.location.str.replace("西北师范学院汉语言文学专业学","甘肃")
df.location = df.location.str.replace("人民日报社副总编辑","北京")
df.location = df.location.str.replace("德国海德堡大学理论医学","德国")
df.location = df.location.str.replace("南开大学","天津")
df.location = df.location.str.replace("武汉理工大学管理学院管理科学职研究生学管理学博士学","湖北")
df.location = df.location.str.replace("厦门大学","福建")
df.location = df.location.str.replace("中山大学经济学政治经济学","广东")
df.location = df.location.str.replace("航天总公司二三〇厂副厂长","北京")
df.location = df.location.str.replace("日本大","日本")
df.location = df.location.str.replace("法国巴黎第七大学圣·路易医院中心实验室外籍住院","法国")
df.location = df.location.str.replace("第二炮兵五十四基地副参谋","解放军")
df.location = df.location.str.replace("航天总公司人事劳动教育局负责人主持工","北京")
df.location = df.location.str.replace("上海建筑学专业学","上海")
df.location = df.location.str.replace("北京植物生理学与生物化学","北京")
df.location = df.location.str.replace("光明日报社总编室主","北京")
df.location = df.location.str.replace("昌都解放委员会第一副主任","解放军")
df.location = df.location.str.replace("航天总公司二三〇厂","北京")
df.location = df.location.str.replace("光明日报社总","北京")
df.location = df.location.str.replace("东北师范大学","吉林")
df.location = df.location.str.replace("华东化工学院化工热力学","上海")
df.location = df.location.str.replace("西北轻工学院硅酸盐专业学","陕西")
df.location = df.location.str.replace("北京研究生院职研究生课程班学","北京")
df.location = df.location.str.replace("上海院","上海")
df.location = df.location.str.replace("西南政法学院","重庆")
df.location = df.location.str.replace("北京研究所第四纪地质学","北京")
df.location = df.location.str.replace("北京财政金融专业学","北京")
df.location = df.location.str.replace("河南政治经济学专业学","河南")
df.location = df.location.str.replace("南昌大学历史学","江西")
df.location = df.location.str.replace("北京二分校工业经济管理专业学","北京")
df.location = df.location.str.replace("上海理论研究所实验力学","上海")
df.location = df.location.str.replace("华东化工学院化学工程专业学","上海")
df.location = df.location.str.replace("中国医学科学院础所微生物及免疫学","北京")
df.location = df.location.str.replace("武汉汽车工业大学工商管理学院经济学经济学硕士学","湖北")
df.location = df.location.str.replace("北京植物生理学与生物化学国家自然科学基金委生命科学","北京")
df.location = df.location.str.replace("云南","云南")
df.location = df.location.str.replace("上海管理学院职研究生学理学硕士学","上海")
df.location = df.location.str.replace("东北师范大学经济学专业学","吉林")
df.location = df.location.str.replace("上海上海","上海")
df.location = df.location.str.replace("吉林经济学专业学","吉林")
df.location = df.location.str.replace("江西历史学","江西")
df.location = df.location.str.replace("北京职研究生课程班学","北京")
df.location = df.location.str.replace("北京国家自然科学基金委生命科学","北京")


In [228]:
df.location.value_counts()

北京       544
上海        91
浙江        86
福建        63
解放军       50
西藏        49
山东        46
辽宁        45
吉林        39
河南        29
河北        26
安徽        26
湖北        23
青海        23
黑龙江       21
天津        21
广西        20
香港        18
新疆维吾尔     18
陕西        17
江西        16
甘肃        16
内蒙古       16
宁夏        15
广东        15
贵州        11
美国        10
澳门         8
湖南         8
重庆         7
四川         5
江苏         5
山西         4
德国         3
加拿大        2
英国         2
日本         2
海南         1
法国         1
台湾         1
云南         1
Name: location, dtype: int64

In [214]:
df_test = df

In [215]:
df_test["year_clean"] = pd.to_datetime(df_test.year_clean,format='%Y')

In [216]:
df_test.drop(columns = ["year"], inplace = True)

In [217]:
df_test = df_test.groupby("url").apply(lambda x : x.drop_duplicates("year_clean")
                                          .set_index('year_clean')
                                          .resample('YS')
                                          .ffill())

In [222]:
df_test.head()

content  \
url                                                         year_clean                             
http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html 1969-01-01  陕西省延川县文安驿公社梁家河大队知青、党支部书记   
                                                            1970-01-01  陕西省延川县文安驿公社梁家河大队知青、党支部书记   
                                                            1971-01-01  陕西省延川县文安驿公社梁家河大队知青、党支部书记   
                                                            1972-01-01  陕西省延川县文安驿公社梁家河大队知青、党支部书记   
                                                            1973-01-01  陕西省延川县文安驿公社梁家河大队知青、党支部书记   

                                                                       name  \
url                                                         year_clean        
http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html 1969-01-01  习近平   
                                                            1970-01-01  习近平   
                                                            1971-01-01  习近平   
                                                            1972-01-01  习近平   
                                                            1973-01-01  习近平   

                                                                                                                                url  \
url                                                         year_clean                                                                
http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html 1969-01-01  http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html   
                                                            1970-01-01  http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html   
                                                            1971-01-01  http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html   
                                                            1972-01-01  http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html   
                                                            1973-01-01  http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html   

                                                                       location  
url                                                         year_clean           
http://cpc.people.com.cn/n1/2017/1025/c414940-29608803.html 1969-01-01       陕西  
                                                            1970-01-01       陕西  
                                                            1971-01-01       陕西  
                                                            1972-01-01       陕西  
                                                            1973-01-01       陕西

In [226]:
df_test = df_test.reset_index(level=["year_clean"])

In [227]:
df_test.to_csv("pl_resume_clean.csv", index = False)